# 🧠 Classificação e Segmentação de Imagens de Ultrassom da Mama
Usando VGG-16 (classificação) e U-Net (segmentação) com o dataset do Kaggle.

In [4]:
%pip install tensorflow opencv-python scikit-learn matplotlib kaggle

  Using cached bleach-6.2.0-py3-none-any.whl.metadata (30 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached bleach-6.2.0-py3-none-any.whl (163 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)

   ------------- -------------------------- 2/6 [tqdm]
   ------------- -------------------------- 2/6 [tqdm]
   ------------- -------------------------- 2/6 [tqdm]
   ------------- -------------------------- 2/6 [tqdm]
   -------------------- ------------------- 3/6 [python-slugify]
   -------------------------- ------------- 4/6 [bleach]
   -------------------------- ------------- 4/6 [bleach]
   -------------------------- ------------- 4/6 [bleach]
   -------------------------- ------------- 4/6 [bleach]
   --------------------------------- ------ 5/6 [kaggle]
   --------------------------------- ------ 5/6 [kaggle]
   --------------------------------- ------ 5/6 [kaggle]
   --------------------------------- ------ 5/6 [kaggle]
   -------


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 📁 Importação de Bibliotecas

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
import tensorflow as tf

## 🗂️ Carregamento e Pré-processamento para Classificação

In [3]:
base_path = 'Breast Ultrasound Images Dataset/Dataset_BUSI_with_GT'
classes = ['benign', 'malignant', 'normal']
img_size = 224

X = []
y = []

for idx, label in enumerate(classes):
    folder = os.path.join(base_path, label)
    for filename in os.listdir(folder):
        if 'mask' not in filename:
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (img_size, img_size))
            X.append(img)
            y.append(idx)

X = np.array(X) / 255.0
y = to_categorical(y, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'Breast Ultrasound Images Dataset/Dataset_BUSI_with_GT\\benign'

## 🧠 Modelo VGG-16 para Classificação

In [ ]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
for layer in vgg.layers:
    layer.trainable = False

x = vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model_class = Model(inputs=vgg.input, outputs=predictions)
model_class.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_class.summary()

## 📊 Treinamento do Modelo de Classificação

In [ ]:
model_class.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=16)

## 🖼️ Função para Carregar Imagens e Máscaras (Segmentação)

In [ ]:
def load_images_and_masks(class_folder):
    img_list = []
    mask_list = []
    folder = os.path.join(base_path, class_folder)
    for fname in os.listdir(folder):
        if 'mask' not in fname:
            img_path = os.path.join(folder, fname)
            mask_path = os.path.join(folder, fname.split('.')[0] + '_mask.png')
            if not os.path.exists(mask_path): continue
            img = cv2.imread(img_path)
            img = cv2.resize(img, (128, 128))
            img = img / 255.0
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (128, 128))
            mask = np.expand_dims(mask, axis=-1) / 255.0
            img_list.append(img)
            mask_list.append(mask)
    return img_list, mask_list

## 📥 Carregamento dos Dados de Segmentação

In [ ]:
X_seg, y_seg = [], []
for c in ['benign', 'malignant']:
    imgs, masks = load_images_and_masks(c)
    X_seg += imgs
    y_seg += masks

X_seg = np.array(X_seg)
y_seg = np.array(y_seg)

X_train_seg, X_val_seg, y_train_seg, y_val_seg = train_test_split(X_seg, y_seg, test_size=0.2, random_state=42)

## 🧬 Modelo U-Net para Segmentação

In [ ]:
def unet_model(input_size=(128,128,3)):
    inputs = Input(input_size)
    c1 = layers.Conv2D(16, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D()(c1)

    c2 = layers.Conv2D(32, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D()(c2)

    c3 = layers.Conv2D(64, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(64, 3, activation='relu', padding='same')(c3)

    u1 = layers.UpSampling2D()(c3)
    u1 = layers.Concatenate()([u1, c2])
    c4 = layers.Conv2D(32, 3, activation='relu', padding='same')(u1)
    c4 = layers.Conv2D(32, 3, activation='relu', padding='same')(c4)

    u2 = layers.UpSampling2D()(c4)
    u2 = layers.Concatenate()([u2, c1])
    c5 = layers.Conv2D(16, 3, activation='relu', padding='same')(u2)
    c5 = layers.Conv2D(16, 3, activation='relu', padding='same')(c5)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c5)

    model = Model(inputs, outputs)
    return model

model_seg = unet_model()
model_seg.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_seg.summary()

## 🧪 Treinamento do Modelo de Segmentação

In [ ]:
model_seg.fit(X_train_seg, y_train_seg, validation_data=(X_val_seg, y_val_seg), epochs=10, batch_size=8)

## 🔍 Visualização de Resultados de Segmentação

In [ ]:
def show_results(model, X, y, index=0):
    pred = model.predict(np.expand_dims(X[index], axis=0))[0]
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(X[index])
    axs[0].set_title("Imagem")
    axs[1].imshow(y[index].squeeze(), cmap='gray')
    axs[1].set_title("Máscara Real")
    axs[2].imshow(pred.squeeze(), cmap='gray')
    axs[2].set_title("Máscara Predita")
    plt.show()

show_results(model_seg, X_val_seg, y_val_seg, index=5)